In [1]:
# Data Manipulation Libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Additional Libraries
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('D:/Automobile Loan Default/data/processed/preprocessed_data.csv', low_memory=False)
pd.set_option('display.max_columns', None)
df.head()

,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Client_Education,Workphone_Working,Client_Family_Members,Default,Age_Years,Employed_Years,Client_Income_Type_Commercial,Client_Income_Type_Govt Job,Client_Income_Type_Maternity leave,Client_Income_Type_Retired,Client_Income_Type_Service,Client_Income_Type_Student,Client_Marital_Status_D,Client_Marital_Status_M,Client_Marital_Status_S,Client_Marital_Status_W,Client_Gender_Female,Client_Gender_Male,Loan_Contract_Type_CL,Loan_Contract_Type_RL
0,0.134199,0,0,1,0,0.000000,0.377680,0.593635,4,0,0.333333,0,0.354167,0.085714,1,0,0,0,0,0,0,1,0,0,0,1,1,0
1,0.567100,1,0,1,1,0.000000,0.071831,0.298446,0,1,0.333333,0,0.375000,0.314286,0,0,0,0,1,0,0,1,0,0,0,1,1,0
2,0.494949,0,0,1,0,0.333333,0.366600,0.476946,1,0,0.333333,0,0.520833,0.400000,0,0,0,0,1,0,0,0,0,1,0,1,1,0
3,1.000000,1,0,1,0,0.666667,0.862669,0.617858,4,0,1.000000,0,0.208333,0.228571,1,0,0,0,0,0,0,1,0,0,1,0,1,0
4,0.278499,0,1,1,1,0.333333,0.061638,0.080772,4,0,0.333333,0,0.354167,0.085714,0,0,0,0,1,0,0,0,0,1,1,0,1,0


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV


X = df.drop('Default', axis=1)  # Features
y = df['Default']                # Target variable

from imblearn.under_sampling import RandomUnderSampler

under=RandomUnderSampler()
X,y=under.fit_resample(X,y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
     

In [18]:
import os
import pandas as pd

# Define the folder path
interim_folder = r"D:\Automobile Loan Default\data\interim"

# Make sure the folder exists
os.makedirs(interim_folder, exist_ok=True)

# Combine X_train and y_train into a single DataFrame
train_df = pd.concat([X_train, y_train], axis=1)

# Combine X_test and y_test into a single DataFrame
test_df = pd.concat([X_test, y_test], axis=1)

# Save as CSV
train_df.to_csv(os.path.join(interim_folder, "train.csv"), index=False)
test_df.to_csv(os.path.join(interim_folder, "test.csv"), index=False)


In [25]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib  # <-- needed for saving

# Define the model
rf_model = RandomForestClassifier()

# Train the model
rf_model.fit(X_train, y_train)

# Evaluate the model
train_accuracy = accuracy_score(y_train, rf_model.predict(X_train))
test_accuracy = accuracy_score(y_test, rf_model.predict(X_test))

print("Random Forest Classifier:")
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

#save the trained model
joblib.dump(rf_model,'D:/Automobile Loan Default/models/rf_model_weighted.pkl')
print("\n Model saved successfully")


Random Forest Classifier:
Training Accuracy: 1.0
Testing Accuracy: 0.6090909090909091
Confusion Matrix:
 [[920 565]
 [596 889]]
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.62      0.61      1485
           1       0.61      0.60      0.60      1485

    accuracy                           0.61      2970
   macro avg       0.61      0.61      0.61      2970
weighted avg       0.61      0.61      0.61      2970


 Model saved successfully


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Define the model
rf_model_hyper = RandomForestClassifier()

# Define hyperparameters for tuning
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 15, 20],
    'min_samples_split': [2, 5, 10]
}

# Perform GridSearchCV
grid_search_rf = GridSearchCV(rf_model_hyper, param_grid, cv=5)
grid_search_rf.fit(X_train, y_train)

# Print the best hyperparameters
best_hyperparameters = grid_search_rf.best_params_
print("Best Hyperparameters (Random Forest):", best_hyperparameters)

# Evaluate the model
train_accuracy = accuracy_score(y_train, grid_search_rf.predict(X_train))
test_accuracy = accuracy_score(y_test, grid_search_rf.predict(X_test))

print("Random Forest Classifier:")
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Make predictions on the test set using the best model found by GridSearchCV
y_pred = grid_search_rf.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)
     

Best Hyperparameters (Random Forest): {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
Random Forest Classifier:
Training Accuracy: 0.7554302071055733
Testing Accuracy: 0.6134680134680135
Confusion Matrix:
 [[893 592]
 [556 929]]
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.60      0.61      1485
           1       0.61      0.63      0.62      1485

    accuracy                           0.61      2970
   macro avg       0.61      0.61      0.61      2970
weighted avg       0.61      0.61      0.61      2970



In [8]:
#Support Vector Classification (SVC)
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the model
svm_model = SVC()

# Train the model
svm_model.fit(X_train, y_train)

# Evaluate the model
train_accuracy = accuracy_score(y_train, svm_model.predict(X_train))
test_accuracy = accuracy_score(y_test, svm_model.predict(X_test))

print("SVM Classifier:")
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Make predictions on the test set
y_pred = svm_model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

SVM Classifier:
Training Accuracy: 0.6103721165179323
Testing Accuracy: 0.6037037037037037
Confusion Matrix:
 [[820 665]
 [512 973]]
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.55      0.58      1485
           1       0.59      0.66      0.62      1485

    accuracy                           0.60      2970
   macro avg       0.60      0.60      0.60      2970
weighted avg       0.60      0.60      0.60      2970



In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Define the model
svm_model_hyper = SVC()

# Define hyperparameters for tuning
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
}

# Perform GridSearchCV
grid_search_svm = GridSearchCV(svm_model_hyper, param_grid, cv=5)
grid_search_svm.fit(X_train, y_train)

# Print the best hyperparameters
best_hyperparameters = grid_search_svm.best_params_
print("Best Hyperparameters (SVM):", best_hyperparameters)

# Evaluate the model
train_accuracy = accuracy_score(y_train, grid_search_svm.predict(X_train))
test_accuracy = accuracy_score(y_test, grid_search_svm.predict(X_test))

print("SVM Classifier:")
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Make predictions on the test set using the best model found by GridSearchCV
y_pred = grid_search_svm.predict(X_test)

# Compute the confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Best Hyperparameters (SVM): {'C': 1, 'kernel': 'rbf'}
SVM Classifier:
Training Accuracy: 0.6103721165179323
Testing Accuracy: 0.6037037037037037
Confusion Matrix:
 [[820 665]
 [512 973]]
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.55      0.58      1485
           1       0.59      0.66      0.62      1485

    accuracy                           0.60      2970
   macro avg       0.60      0.60      0.60      2970
weighted avg       0.60      0.60      0.60      2970



In [10]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Define the model
nb_model = GaussianNB()

# Fit the model
nb_model.fit(X_train, y_train)

# Evaluate the model
train_accuracy = accuracy_score(y_train, nb_model.predict(X_train))
test_accuracy = accuracy_score(y_test, nb_model.predict(X_test))

print("Naive Bayes Classifier:")
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Make predictions on the test set
y_pred = nb_model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Naive Bayes Classifier:
Training Accuracy: 0.5684458663074592
Testing Accuracy: 0.5747474747474748
Confusion Matrix:
 [[ 605  880]
 [ 383 1102]]
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.41      0.49      1485
           1       0.56      0.74      0.64      1485

    accuracy                           0.57      2970
   macro avg       0.58      0.57      0.56      2970
weighted avg       0.58      0.57      0.56      2970



In [11]:
from sklearn.naive_bayes import GaussianNB

# Define the model
nb_model_hyper = GaussianNB()

# No hyperparameters to tune for Gaussian Naive Bayes

# Fit the model
nb_model_hyper.fit(X_train, y_train)

# Evaluate the model
train_accuracy = accuracy_score(y_train, nb_model_hyper.predict(X_train))
test_accuracy = accuracy_score(y_test, nb_model_hyper.predict(X_test))

print("Naive Bayes Classifier:")
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Make predictions on the test set
y_pred = nb_model_hyper.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Naive Bayes Classifier:
Training Accuracy: 0.5684458663074592
Testing Accuracy: 0.5747474747474748
Confusion Matrix:
 [[ 605  880]
 [ 383 1102]]
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.41      0.49      1485
           1       0.56      0.74      0.64      1485

    accuracy                           0.57      2970
   macro avg       0.58      0.57      0.56      2970
weighted avg       0.58      0.57      0.56      2970



In [23]:
#Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Define the model
gb_model = GradientBoostingClassifier()

# Fit the model
gb_model.fit(X_train, y_train)

# Evaluate the model
train_accuracy = accuracy_score(y_train, gb_model.predict(X_train))
test_accuracy = accuracy_score(y_test, gb_model.predict(X_test))

print("Gradient Boosting Classifier:")
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Make predictions on the test set
y_pred = gb_model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)



Gradient Boosting Classifier:
Training Accuracy: 0.637986192961778
Testing Accuracy: 0.6195286195286195
Confusion Matrix:
 [[910 575]
 [555 930]]
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.61      0.62      1485
           1       0.62      0.63      0.62      1485

    accuracy                           0.62      2970
   macro avg       0.62      0.62      0.62      2970
weighted avg       0.62      0.62      0.62      2970



In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Define the model
gb_model_hyper = GradientBoostingClassifier()

# Define hyperparameters for tuning
param_grid = {
    'n_estimators': [10, 50, 100],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# Perform GridSearchCV
grid_search_gb = GridSearchCV(gb_model_hyper, param_grid, cv=5)
grid_search_gb.fit(X_train, y_train)

# Print the best hyperparameters
best_hyperparameters = grid_search_gb.best_params_
print("Best Hyperparameters (GradientBoostingClassifier):", best_hyperparameters)

# Evaluate the model using GridSearchCV results
train_accuracy = accuracy_score(y_train, grid_search_gb.predict(X_train))
test_accuracy = accuracy_score(y_test, grid_search_gb.predict(X_test))

print("Gradient Boosting Classifier:")
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Make predictions on the test set using the best model found by GridSearchCV
y_pred = grid_search_gb.predict(X_test)

# Compute the confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Best Hyperparameters (GradientBoostingClassifier): {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}
Gradient Boosting Classifier:
Training Accuracy: 0.668883650446203
Testing Accuracy: 0.6138047138047138
Confusion Matrix:
 [[891 594]
 [553 932]]
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.60      0.61      1485
           1       0.61      0.63      0.62      1485

    accuracy                           0.61      2970
   macro avg       0.61      0.61      0.61      2970
weighted avg       0.61      0.61      0.61      2970



In [14]:
#Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Define the model
dt_model = DecisionTreeClassifier()

# Fit the model
dt_model.fit(X_train, y_train)

# Evaluate the model
train_accuracy = accuracy_score(y_train, dt_model.predict(X_train))
test_accuracy = accuracy_score(y_test, dt_model.predict(X_test))

print("Decision Tree Classifier:")
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Make predictions on the test set
y_pred = dt_model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Decision Tree Classifier:
Training Accuracy: 0.9999158107425492
Testing Accuracy: 0.5397306397306397
Confusion Matrix:
 [[766 719]
 [648 837]]
Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.52      0.53      1485
           1       0.54      0.56      0.55      1485

    accuracy                           0.54      2970
   macro avg       0.54      0.54      0.54      2970
weighted avg       0.54      0.54      0.54      2970



In [15]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Define the model
dt_model_hyper = DecisionTreeClassifier()

# Define hyperparameters for tuning
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform GridSearchCV
grid_search_dc = GridSearchCV(dt_model_hyper, param_grid, cv=5)
grid_search_dc.fit(X_train, y_train)

# Print the best hyperparameters
best_hyperparameters = grid_search_dc.best_params_
print("Best Hyperparameters (Decision Tree):", best_hyperparameters)

# Get the best model
best_dt_model = grid_search_dc.best_estimator_

# Evaluate the model
train_accuracy = accuracy_score(y_train, best_dt_model.predict(X_train))
test_accuracy = accuracy_score(y_test, best_dt_model.predict(X_test))

print("Decision Tree Classifier:")
print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)

# Make predictions on the test set
y_pred = best_dt_model.predict(X_test)

# Compute the confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Best Hyperparameters (Decision Tree): {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}
Decision Tree Classifier:
Training Accuracy: 0.6716618959420778
Testing Accuracy: 0.5727272727272728
Confusion Matrix:
 [[808 677]
 [592 893]]
Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.54      0.56      1485
           1       0.57      0.60      0.58      1485

    accuracy                           0.57      2970
   macro avg       0.57      0.57      0.57      2970
weighted avg       0.57      0.57      0.57      2970

